**Bài tập 1 và 2 thuộc đề 1**

- Bài tập 1: Cho 1 bảng dữ liệu, yêu cầu:
  - Chuẩn hoá dữ liệu
  - Dự đoán kết quả (0 or 1) của tập value cho trước (cần chuẩn hoá tập value phù hợp với pp được chọn ở ý 1)
  - Sử dụng thuật toán Naive Bayes để dự đoán kết quả (0 or 1) của tập value cho trước


- Chuẩn hoá

In [33]:
import pandas as pd
from sklearn.preprocessing import MinMaxScaler

data = {
    'ID': [1, 2, 3, 4, 5],
    'Age': [25, 30, 35, 20, 40],
    'Income': [5000, 7000, 9000, 3000, 10000],
    'Student': ['No', 'Yes', 'Yes', 'No', 'Yes'],
    'Credit_Rating': [0.75, 0.80, 0.85, 0.70, 0.90],
    'Buy_Computer': ['Yes', 'No', 'Yes', 'No', 'Yes']
}
df = pd.DataFrame(data)
# Mã hóa biến Student thành số (dummy variable)
df['Student'] = df['Student'].apply(lambda x: 1 if x == 'Yes' else 0)

# Áp dụng MinMaxScaler để chuẩn hóa dữ liệu
min_max = MinMaxScaler()
columns_to_scale = ['Age', 'Income', 'Credit_Rating']
df_scaled = df.copy()  # Tạo một bản sao của DataFrame gốc

df_scaled[columns_to_scale] = min_max.fit_transform(df[columns_to_scale]).round(2)

df_scaled

,ID,Age,Income,Student,Credit_Rating,Buy_Computer
0,1,0.25,0.29,0,0.25,Yes
1,2,0.50,0.57,1,0.50,No
2,3,0.75,0.86,1,0.75,Yes
3,4,0.00,0.00,0,0.00,No
4,5,1.00,1.00,1,1.00,Yes


- Dự đoán kết quả của tập value
  - Mã hoá tập value
  - Tính khoảng cách (Euclide hoặc Cosine)

In [35]:
# Dữ liệu của điểm dữ liệu mới
new_data = {
    'ID': 6,
    'Age': 20,
    'Income': 8000,
    'Student': 1,
    'Credit_Rating': 0.85
}

# Thêm dữ liệu mới vào DataFrame
df_new = pd.DataFrame([new_data])

# Sử dụng phương thức pd.concat() để chèn dữ liệu vào df gốc
df_updated = pd.concat([df, df_new], ignore_index=True)

#chuẩn hoá lại toàn bộ dataframe (df_updated), sử dụng pp min-max
# Áp dụng MinMaxScaler để chuẩn hóa dữ liệu
min_max = MinMaxScaler()
df_updated[columns_to_scale] = min_max.fit_transform(df_updated[columns_to_scale]).round(2)
df_updated

,ID,Age,Income,Student,Credit_Rating,Buy_Computer
0,1,0.25,0.29,0,0.25,Yes
1,2,0.50,0.57,1,0.50,No
2,3,0.75,0.86,1,0.75,Yes
3,4,0.00,0.00,0,0.00,No
4,5,1.00,1.00,1,1.00,Yes
5,6,0.00,0.71,1,0.75,NaN


- tính toán khoảng cách


In [39]:
#bằng công thức cosine
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity

# Chuyển đổi DataFrame thành mảng numpy để sử dụng cosine_similarity
X_train = df_scaled[['Age', 'Income', 'Student', 'Credit_Rating']].values
X_new = [0,0.71,1, 0.75]

X_new_reshaped = np.array(X_new).reshape(1, -1)

# Tính toán khoảng cách cosine
cosine_distances = 1 - cosine_similarity(X_train, X_new_reshaped)

# Hiển thị khoảng cách cosine
print("Khoảng cách cosine:")
print(cosine_distances)

df_scaled['Cosine_Distance'] = cosine_distances

# Sắp xếp các hàng theo khoảng cách tăng dần
df_sorted = df_scaled.sort_values(by='Cosine_Distance')

# Lấy K hàng đầu tiên
K = 3
nearest_neighbors = df_sorted.head(K)

# Hiển thị các hàng láng giềng gần nhất
print(f"{K} hàng láng giềng gần nhất:")
print(nearest_neighbors)

Khoảng cách cosine:
[[0.40154827]
 [0.08357052]
 [0.10686007]
 [1.        ]
 [0.14438794]]
3 hàng láng giềng gần nhất:
   ID   Age  Income  Student  Credit_Rating Buy_Computer  Cosine_Distance
1   2  0.50    0.57        1           0.50           No         0.083571
2   3  0.75    0.86        1           0.75          Yes         0.106860
4   5  1.00    1.00        1           1.00          Yes         0.144388


In [45]:
# Tính toán khoảng cách Euclid
euclidean_distances = np.sqrt(np.sum((X_train - X_new) ** 2, axis=1))

# Thêm cột khoảng cách vào DataFrame
df_scaled['Euclidean_Distance'] = euclidean_distances

In [46]:
# Sắp xếp các hàng theo khoảng cách tăng dần
df_sorted = df_scaled.sort_values(by='Euclidean_Distance')

# Lấy K hàng đầu tiên
K = 3
nearest_neighbors = df_sorted.head(K)

print("Dataframe với khoảng cách Euclidean:")
print(df_scaled['Cosine_Distance'])

# Hiển thị các hàng láng giềng gần nhất
print(f"{K} hàng láng giềng gần nhất:")
print(nearest_neighbors[['ID', 'Age', 'Income', 'Student', 'Credit_Rating', 'Buy_Computer', 'Euclidean_Distance']])

Dataframe với khoảng cách Euclidean:
0    0.401548
1    0.083571
2    0.106860
3    1.000000
4    0.144388
Name: Cosine_Distance, dtype: float64
3 hàng láng giềng gần nhất:
   ID   Age  Income  Student  Credit_Rating Buy_Computer  Euclidean_Distance
1   2  0.50    0.57        1           0.50           No            0.576281
2   3  0.75    0.86        1           0.75          Yes            0.764853
4   5  1.00    1.00        1           1.00          Yes            1.070794


Bài tập 2: Sử dụng thuật toán K-Means và phương pháp tính khoảng cách Euclide để xác định cụm

Giải tay (thủ công)

Lần 1:
- Bước 1: Chọn điểm trung tâm của cụm: Tính điểm trung tâm (centroid) của cụm trong thuật toán k-means được tính theo công thức:
$$
m_i = \frac{1}{|C_i|} \sum_{\mathbf{x}_i \in C_i} \mathbf{x}_i
$$  
  > - m_i là điểm trung tâm của cụm C_i.
  > - |C_i| là số lượng điểm trong cụm C_i.
  > - X_i là điểm dữ liệu trong cụm C_i
  > - vd: cho cụm C1 = {(1,2), (2,3), (3,1)}, điểm trung tâm cụm C1: m_c1 = 1/3 * ((1,2) + (2,3) + (3,1)) = 1/3 * (6,6) = (2,2)
- Bước 2: Tính khoảng cách từ các điểm đến trung tâm các cụm, sử dụng công thức tính khoảng cách euclide (hoặc các công thức khác như cosine)
  - d(x, m_i) = ||x-m_i|| 
  - vd: cho 2 cụm C1= {(1,1), (3,3),(4,3)}, cụm C2 = {(1,2),(2,1),(3,4)} 
    => điểm trung tâm cụm C1: m_c1 = (8/3, 7/3), cụm C2: m_c2 = (2, 7/3)
    + từ điểm (1,1) tới điểm trung tâm c1: 2.13
    + từ điểm (1,1) tới điểm trung tâm c2: 1.67
    
    => điểm (1,1) thuộc cụm c2

    tương tư với các điểm khác
- Bước 3: Phân cụm lại các điểm

Lần 2: lặp lại các bước ở lần 1 ... cho đến khi không còn sự thay đổi giữa các điểm với các cụm


In [3]:
#Code tính khoảng cách euclide từ các điểm đến điểm trung tâm cụm
import numpy as np
# Hàm tính khoảng cách Euclide giữa các điểm dữ liệu và các trung tâm cụm
def compute_euclidean_distances(data_points, centroids):
    # data_points: ma trận (num_points, num_dimensions)
    # centroids: ma trận (num_clusters, num_dimensions)
    # Tính khoảng cách Euclide cho từng cặp điểm dữ liệu và trung tâm cụm
    distances = np.linalg.norm(data_points[:, np.newaxis, :] - centroids[np.newaxis, :, :], axis=2)
    return distances

# Hàm để tìm cụm gần nhất cho mỗi điểm dữ liệu
def assign_clusters(distances):
    # Tìm chỉ số của cụm gần nhất (min distance)
    return np.argmin(distances, axis=1)

# Ví dụ minh họa
if __name__ == '__main__':
    # Giả sử chúng ta có các điểm dữ liệu trong không gian 2D
    data_points = np.array([
        [1.0, 1.0],
        [1.0, 2.0],
        [2.0, 1.0],
        [3.0, 3.0],
        [3.0, 4.0],
        [4.0, 3.0]
    ])
    # Và có hai trung tâm cụm
    centroids = np.array([
        [8/3, 7/3],
        [2, 7/3]
    ])
    # Tính khoảng cách Euclide
    distances = compute_euclidean_distances(data_points, centroids)
    
    # In ra khoảng cách từ các điểm dữ liệu đến từng trung tâm cụm
    print("Distances:")
    print(distances)    
    # # Xác định cụm gần nhất cho mỗi điểm dữ liệu
    # cluster_assignments = assign_clusters(distances)
    # # In ra kết quả phân cụm
    # print("\nCluster assignments:")
    # print(cluster_assignments)


Distances:
[[2.13437475 1.66666667]
 [1.69967317 1.05409255]
 [1.49071198 1.33333333]
 [0.74535599 1.20185043]
 [1.69967317 1.94365063]
 [1.49071198 2.10818511]]


- **Code để xác định điểm trung tâm và phân cụm**

In [5]:
import numpy as np
# Hàm tính khoảng cách Euclide giữa các điểm dữ liệu và các trung tâm cụm
def compute_euclidean_distances(data_points, centroids):
    # data_points: ma trận (num_points, num_dimensions)
    # centroids: ma trận (num_clusters, num_dimensions)
    # Tính khoảng cách Euclide cho từng cặp điểm dữ liệu và trung tâm cụm
    distances = np.linalg.norm(data_points[:, np.newaxis, :] - centroids[np.newaxis, :, :], axis=2)
    
    return distances

# Hàm cập nhật trung tâm cụm dựa trên phân cụm hiện tại
def update_centroids(data_points, cluster_assignments, num_clusters):
    new_centroids = np.zeros((num_clusters, data_points.shape[1]))
    for k in range(num_clusters):
        # Lấy các điểm dữ liệu thuộc cụm k
        cluster_points = data_points[cluster_assignments == k]
        # Tính trung bình của các điểm dữ liệu này để cập nhật trung tâm cụm k
        if len(cluster_points) > 0:
            new_centroids[k] = np.mean(cluster_points, axis=0)
    return new_centroids

# Hàm phân cụm K-means
def kmeans(data_points, initial_centroids, max_iters=100):
    num_clusters = initial_centroids.shape[0]
    centroids = initial_centroids
    
    for i in range(max_iters):
        # Bước 1: Tính khoảng cách từ các điểm dữ liệu đến các trung tâm cụm hiện tại
        distances = compute_euclidean_distances(data_points, centroids)
        
        # Bước 2: Gán lại các điểm dữ liệu vào các cụm dựa trên khoảng cách ngắn nhất
        cluster_assignments = np.argmin(distances, axis=1)
        
        # In kết quả sau mỗi lần lặp
        print(f"Iteration {i + 1}:")
        print("Centroids:")
        print(centroids)
        print("Cluster assignments:")
        print(cluster_assignments)
        print("------------------------")
        
        # Bước 3: Cập nhật lại các trung tâm cụm dựa trên phân cụm hiện tại
        new_centroids = update_centroids(data_points, cluster_assignments, num_clusters)
        
        # Bước 4: Kiểm tra điều kiện dừng (ví dụ: trung tâm cụm không thay đổi đáng kể)
        if np.allclose(new_centroids, centroids):
            break
        
        # Cập nhật trung tâm cụm mới
        centroids = new_centroids
    
    return centroids, cluster_assignments

# Ví dụ minh họa
if __name__ == '__main__':
    data_points = np.array([
        [1.0, 1.0],
        [1.0, 2.0],
        [2.0, 1.0],
        [3.0, 3.0],
        [3.0, 4.0],
        [4.0, 3.0]
    ])
    
    # Khởi tạo các trung tâm cụm ban đầu (có thể tự tính tay (cách tính ở bài 2 -> bước 1))
    initial_centroids = np.array([
        [8/3, 7/3],
        [2.0, 7/3]
    ])
    
    # Chạy thuật toán K-means với số lần lặp tối đa là 3
    max_iters = 3
    final_centroids, cluster_assignments = kmeans(data_points, initial_centroids, max_iters=max_iters)
    
    # In ra kết quả cuối cùng
    print("\nFinal centroids:")
    print(final_centroids)
    print("\nFinal cluster assignments after", max_iters, "iterations:")
    print(cluster_assignments)


Iteration 1:
Centroids:
[[2.66666667 2.33333333]
 [2.         2.33333333]]
Cluster assignments:
[1 1 1 0 0 0]
------------------------
Iteration 2:
Centroids:
[[3.33333333 3.33333333]
 [1.33333333 1.33333333]]
Cluster assignments:
[1 1 1 0 0 0]
------------------------

Final centroids:
[[3.33333333 3.33333333]
 [1.33333333 1.33333333]]

Final cluster assignments after 3 iterations:
[1 1 1 0 0 0]
